In [1]:
import geopandas as gpd
import pandas as pd

In [2]:
# 读取小区数据
communities = pd.read_csv('communities_with_distance_river.csv')
# 创建小区GeoDataFrame
community_gdf = gpd.GeoDataFrame(communities, geometry=gpd.points_from_xy(communities.longitude, communities.latitude), crs = 'EPSG:4490')

In [3]:
community_gdf.head()

,小区名字,小区地址,均价,容积率,有无停车位,物业费_元/平方米/月,绿化率_percent,latitude,longitude,BUS,HOS,MAR,SUB,PSC,MSC,PAR,CBD,SCBD,RIV,geometry
0,盈彩美居,天河-东圃-盈溪路,54560,2.88,1,1.20,30.0,23.124325,113.415697,256.509826,1535.484711,175.643977,1196.714612,417.639070,856.055562,161.120353,9319.496930,7467.040911,2068.596693,POINT (113.41570 23.12433)
1,兰亭盛荟,天河-东圃-东圃明镜路,70093,3.80,1,3.50,62.0,23.124325,113.415697,256.509826,1535.484711,175.643977,1196.714612,417.639070,856.055562,161.120353,9319.496930,7467.040911,2068.596693,POINT (113.41570 23.12433)
2,中海康城花园,天河-黄村-广园快速路,45544,1.74,1,2.37,40.0,23.132456,113.419568,152.513554,2100.395942,451.259944,1207.328398,388.567987,383.483784,703.540475,9796.623781,8166.680740,3059.158674,POINT (113.41957 23.13246)
3,宝翠园,天河-龙洞-宝翠路4号,58641,2.66,1,1.90,45.2,23.196825,113.369955,23.832257,5297.997351,346.016106,713.996616,423.282983,382.525096,362.763426,9674.438398,9123.633151,9463.923434,POINT (113.36996 23.19682)
4,东方新世界,天河-天河公园-中山大道西158号,84828,3.99,1,2.30,42.0,23.139163,113.366326,219.307358,265.723923,300.167413,1482.307992,433.038452,1132.855629,1241.665762,4742.191821,4540.610699,3082.062562,POINT (113.36633 23.13916)


In [4]:
# 读取购物类poi
shopping = pd.read_csv('poi/shopping.csv')
# 创建购物类poi的GeoDataFrame
shopping_gdf = gpd.GeoDataFrame(shopping, geometry = gpd.points_from_xy(shopping.gcj02_x, shopping.gcj02_y), crs = 'EPSG:4490')

In [5]:
shopping_gdf.head()

,id,name,gcj02_x,gcj02_y,adname,中类,小类,geometry
0,B0J23Z29A0,啊莹小李车标便利店,113.254520,23.113277,越秀区,便民商店/便利店,便民商店/便利店,POINT (113.25452 23.11328)
1,B0H6DS77CV,AG便利店(金天成印刷包装城店),113.251626,23.059602,荔湾区,便民商店/便利店,便民商店/便利店,POINT (113.25163 23.05960)
2,B00141V4YB,华润万家便利超市(北瑜翠园店),113.364408,23.211069,天河区,便民商店/便利店,便民商店/便利店,POINT (113.36441 23.21107)
3,B0HUY5T81E,乐百惠生活超市,113.365122,23.215073,天河区,超级市场,超市,POINT (113.36512 23.21507)
4,B0JRP1Y4ON,8836乐百惠便利店,113.365217,23.215076,天河区,便民商店/便利店,便民商店/便利店,POINT (113.36522 23.21508)


In [6]:
shopping_gdf['中类'].unique()

array(['便民商店/便利店', '超级市场', '综合市场', '商场', '特色商业街'], dtype=object)

In [7]:
# 定义不同类别的商业业态面积（假设面积单位为平方米）
area_by_type = { '商场': 15000,
                 '特色商业街': 5000,
                 '超级市场': 3000,
                '综合市场': 2000,
                '便民商店/便利店': 100
                 }

In [8]:
# 遍历每个小区，计算800米缓冲区内的总商业面积
total_commercial_area_within_800m = []
for community in community_gdf.itertuples():
    buffer = community.geometry.buffer(800 / 111320) # 111.32公里大约等于1度
    poi_within_buffer = shopping_gdf[shopping_gdf.geometry.within(buffer)]
    total_area = 0
    for poi in poi_within_buffer.itertuples():
        total_area += area_by_type.get(poi.中类, 0)
    total_commercial_area_within_800m.append(total_area)
communities['ARE'] = total_commercial_area_within_800m

In [9]:
communities['ARE'].describe()

count      2282.000000
mean     282679.316389
std      149399.526299
min           0.000000
25%      186550.000000
50%      256000.000000
75%      378900.000000
max      824700.000000
Name: ARE, dtype: float64

In [10]:
# 新变量写入数据集
communities.to_csv('communities_with_distance_river_area.csv', index=False)